# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
data_df = pd.read_csv('data/AviationData.csv', encoding="latin1")

print(data_df.isna().sum())
print(data_df.info())
print(data_df.describe())

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publication.Date          13771
dtype: i

/var/folders/xs/_w4zn5t54vv8bvs6knszrf3m0000gp/T/ipykernel_84841/181663341.py:1: DtypeWarning: Columns (0: Latitude, 1: Longitude, 2: Broad.phase.of.flight) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv('data/AviationData.csv', encoding="latin1")


In [3]:
state_codes_df = pd.read_csv('data/USState_Codes.csv')
print(state_codes_df.isna().sum())
print(state_codes_df.info())
print(state_codes_df.describe())

US_State        0
Abbreviation    0
dtype: int64
<class 'pandas.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   US_State      62 non-null     str  
 1   Abbreviation  62 non-null     str  
dtypes: str(2)
memory usage: 1.1 KB
None
       US_State Abbreviation
count        62           62
unique       62           62
top     Alabama           AL
freq          1            1


### Inspection Observations

**Shape:** 88,889 rows × 31 columns

**Missingness highlights:**
| Column | % Missing | Notes |
|---|---|---|
| `Schedule` | 86% | Too sparse to be useful |
| `Air.carrier` | 81% | Too sparse to be useful |
| `FAR.Description` | 64% | Regulatory classification — mostly missing |
| `Aircraft.Category` | 64% | Vehicle type — mostly missing |
| `Latitude` / `Longitude` | ~61% | Location data mostly absent |
| `Airport.Code` / `Airport.Name` | 44% / 41% | Partial coverage |
| `Total.Fatal/Serious/Minor.Injuries` | ~13% | Will impute with 0 (no injuries reported) |
| `Total.Uninjured` | 7% | Will impute with 0 |

**Data type issues:**
- `Event.Date` is stored as `str` — must convert to `datetime` before filtering to 1983+
- `Latitude` / `Longitude` flagged as mixed types (numeric strings + NaN) — will be dropped
- Injury columns are `float64` due to NaN presence

**Consistency issues:**
- `Make` column has duplicate entries from inconsistent casing (e.g., `"Cessna"` vs `"CESSNA"`) — will standardize with `.str.title()`
- `Weather.Condition` has both `"UNK"` and `"Unk"` as unknown markers — will consolidate

**`USState_Codes`:** Clean — 62 rows, no missing values, 2 columns (`US_State`, `Abbreviation`).

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [4]:
# Inspect relevant columns before filtering
print("Investigation.Type value counts:")
print(data_df['Investigation.Type'].value_counts(), "\n")

print("Amateur.Built value counts:")
print(data_df['Amateur.Built'].value_counts(dropna=False), "\n")

# Convert Event.Date to datetime for date-based filtering
data_df['Event.Date'] = pd.to_datetime(data_df['Event.Date'])

print(f"Shape before filtering: {data_df.shape}")

# Filter 1: 1983 onwards (40-year max aircraft lifetime per client requirement)
data_df = data_df[data_df['Event.Date'] >= '1983-01-01']
print(f"Shape after date filter (1983+): {data_df.shape}")

# Filter 2: Professional builds only (exclude amateur-built aircraft)
data_df = data_df[data_df['Amateur.Built'] == 'No']
print(f"Shape after Amateur.Built filter: {data_df.shape}")

# Filter 3: Accidents only (exclude incidents — incidents are minor and don't reflect safety outcomes)
data_df = data_df[data_df['Investigation.Type'] == 'Accident']
print(f"Shape after Investigation.Type filter (Accidents only): {data_df.shape}")

Investigation.Type value counts:
Investigation.Type
Accident    85015
Incident     3874
Name: count, dtype: int64 

Amateur.Built value counts:
Amateur.Built
No     80312
Yes     8475
NaN      102
Name: count, dtype: int64 

Shape before filtering: (88889, 31)
Shape after date filter (1983+): (85289, 31)
Shape after Amateur.Built filter: (76960, 31)
Shape after Investigation.Type filter (Accidents only): (73286, 31)


**Filtering decisions:**
- **Date (1983+):** Client specified a 40-year max aircraft lifetime, so records prior to 1983 are excluded as they reflect retired models.
- **Amateur.Built == 'No':** Client is only interested in professional/commercial aircraft. The ~102 NaN rows are also excluded since their build status is unknown.
- **Investigation.Type == 'Accident':** Incidents are minor events that don't result in meaningful damage or injury outcomes. Including them would dilute safety metrics. Only ~4% of records are incidents, so this has minimal impact on sample size.

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

**Cleaning assumptions:**
- NaN values in all four injury columns (`Total.Fatal.Injuries`, `Total.Serious.Injuries`, `Total.Minor.Injuries`, `Total.Uninjured`) are imputed with **0**. Missing counts most likely indicate no injuries were recorded, not that the data is truly absent.
- `Total.Passengers` is estimated as the sum of all four columns — this is the best available proxy for occupants on board since actual passenger manifests are not in the dataset.
- `Injury.Rate` = (`Total.Fatal.Injuries` + `Total.Serious.Injuries`) / `Total.Passengers`. Records where `Total.Passengers == 0` (no occupant data at all) are set to `NaN` to avoid division by zero and misleading rates.

In [5]:
injury_cols = ['Total.Fatal.Injuries', 'Total.Serious.Injuries',
               'Total.Minor.Injuries', 'Total.Uninjured']

# Inspect missing values before imputation
print("NaN counts in injury columns:")
print(data_df[injury_cols].isna().sum(), "\n")

# Impute NaN with 0: missing injury counts most likely mean no injuries were recorded
data_df[injury_cols] = data_df[injury_cols].fillna(0)

# Estimate total passengers on board as sum of all injury/uninjured counts
data_df['Total.Passengers'] = data_df[injury_cols].sum(axis=1)

# Injury.Rate: fraction of passengers with fatal or serious injuries
# Set to NaN where Total.Passengers == 0 (no occupant data recorded) to avoid division by zero
data_df['Injury.Rate'] = np.where(
    data_df['Total.Passengers'] > 0,
    (data_df['Total.Fatal.Injuries'] + data_df['Total.Serious.Injuries']) / data_df['Total.Passengers'],
    np.nan
)

print("Total.Passengers summary:")
print(data_df['Total.Passengers'].describe(), "\n")

print("Injury.Rate summary:")
print(data_df['Injury.Rate'].describe())

NaN counts in injury columns:
Total.Fatal.Injuries       9547
Total.Serious.Injuries    10469
Total.Minor.Injuries       9965
Total.Uninjured            4766
dtype: int64 

Total.Passengers summary:
count    73286.000000
mean         4.507123
std         21.643163
min          0.000000
25%          1.000000
50%          2.000000
75%          2.000000
max        699.000000
Name: Total.Passengers, dtype: float64 

Injury.Rate summary:
count    72852.000000
mean         0.286128
std          0.434153
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: Injury.Rate, dtype: float64


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

**Cleaning assumptions:**
- NaN values in `Aircraft.damage` (~4% of records) are filled with `'Unknown'` — damage status was simply not recorded; treating them as a separate category preserves these records rather than dropping them.
- No casing standardization needed — all values (`Substantial`, `Destroyed`, `Minor`, `Unknown`) are already title case.
- `Is.Destroyed` is a binary derived column: `1` if `Aircraft.damage == 'Destroyed'`, `0` otherwise. This directly captures the client's key concern about total loss events.

In [6]:
# Inspect Aircraft.damage
print("Aircraft.damage value counts (including NaN):")
print(data_df['Aircraft.damage'].value_counts(dropna=False), "\n")

# Fill NaN with 'Unknown' — damage status was not recorded for these events
data_df['Aircraft.damage'] = data_df['Aircraft.damage'].fillna('Unknown')

# Verify no remaining NaNs
print("NaN count after imputation:", data_df['Aircraft.damage'].isna().sum(), "\n")

# Create binary Is.Destroyed column: 1 if the aircraft was fully destroyed, 0 otherwise
data_df['Is.Destroyed'] = (data_df['Aircraft.damage'] == 'Destroyed').astype(int)

print("Is.Destroyed value counts:")
print(data_df['Is.Destroyed'].value_counts())
print(f"\nDestruction rate: {data_df['Is.Destroyed'].mean():.1%}")

Aircraft.damage value counts (including NaN):
Aircraft.damage
Substantial    55651
Destroyed      15462
NaN             1415
Minor            664
Unknown           94
Name: count, dtype: int64 

NaN count after imputation: 0 

Is.Destroyed value counts:
Is.Destroyed
0    57824
1    15462
Name: count, dtype: int64

Destruction rate: 21.1%


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

**Identified cleaning tasks:**
1. **Whitespace** — strip leading/trailing whitespace from all `Make` values
2. **Inconsistent casing** — values like `"CESSNA"` and `"Cessna"` refer to the same manufacturer; standardize to title case to merge duplicates
3. **NaN values** — 63 records missing `Make`; fill with `'Unknown'` to retain the rows
4. **Low sample sizes** — makes with fewer than 50 records provide insufficient data for robust safety comparisons; filter these out

In [7]:
print("Unique Makes before cleaning:", data_df['Make'].nunique())
print("\nSample of inconsistent casing:")
print(sorted(data_df['Make'].dropna().unique())[:20], "\n")

# Task 1 & 2: Strip whitespace and standardize to title case
data_df['Make'] = data_df['Make'].str.strip().str.title()

# Task 3: Fill remaining NaN with 'Unknown'
data_df['Make'] = data_df['Make'].fillna('Unknown')

print("Unique Makes after casing standardization:", data_df['Make'].nunique())

# Task 4: Filter to Makes with >= 50 records
make_counts = data_df['Make'].value_counts()
valid_makes = make_counts[make_counts >= 50].index
data_df = data_df[data_df['Make'].isin(valid_makes)]

print(f"Unique Makes after threshold filter (>=50): {data_df['Make'].nunique()}")
print(f"Shape after Make filtering: {data_df.shape}")
print("\nTop 10 Makes by record count:")
print(data_df['Make'].value_counts().head(10))

Unique Makes before cleaning: 2263

Sample of inconsistent casing:
['1200', '177MF LLC', '2021FX3 LLC', '3XTRIM', 'A. Schleicher GMBH & Co.', 'AAA AIRCRAFT LLC', 'AB SPORTINE AVIACIJA', 'AB Sportine Aviacija', 'ADAMS BALLOONS LLC', 'ADAMS DENNIS ALLEN', 'AERMACCHI', 'AERO ADVENTURE', 'AERO AT SP ZOO', 'AERO COMMANDER', 'AERO SP Z O O', 'AERO TEK INC.', 'AERO VODOCHODY', 'AEROFAB INC', 'AEROFAB INC.', 'AEROLITE'] 

Unique Makes after casing standardization: 1836
Unique Makes after threshold filter (>=50): 83
Shape after Make filtering: (66866, 34)

Top 10 Makes by record count:
Make
Cessna      25410
Piper       13935
Beech        4917
Bell         2537
Boeing       1308
Mooney       1249
Robinson     1186
Grumman      1051
Bellanca      973
Hughes        870
Name: count, dtype: int64


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [8]:
print("NaN count in Model:", data_df['Model'].isna().sum())

# Drop rows with missing Model — no identifier means we can't attribute safety outcomes to a plane type
data_df = data_df.dropna(subset=['Model'])
print(f"Shape after dropping NaN Models: {data_df.shape}\n")

# Standardize Model: strip whitespace and uppercase (models are alphanumeric codes, e.g. '172M', 'PA-28', '737')
data_df['Model'] = data_df['Model'].str.strip().str.upper()

# Check whether model labels are unique to each Make
models_per_make = data_df.groupby('Model')['Make'].nunique()
shared_models = models_per_make[models_per_make > 1]
print(f"Model labels shared across multiple Makes: {len(shared_models)}")
print("\nSample of shared model labels:")
print(shared_models.head(10))

# Models are NOT unique to each Make — create a composite identifier
data_df['Make.Model'] = data_df['Make'] + ' ' + data_df['Model']

print(f"\nUnique Make.Model combinations: {data_df['Make.Model'].nunique()}")
print("\nSample Make.Model values:")
print(data_df['Make.Model'].value_counts().head(10))

NaN count in Model: 15
Shape after dropping NaN Models: (66851, 34)

Model labels shared across multiple Makes: 393

Sample of shared model labels:
Model
100        3
100-180    2
105        2
109        2
109A       2
112        4
112A       3
112TC      2
112TCA     3
114        2
Name: Make, dtype: int64

Unique Make.Model combinations: 5914

Sample Make.Model values:
Make.Model
Cessna 152         2215
Cessna 172         1640
Cessna 172N        1091
Piper PA-28-140     862
Cessna 172M         754
Cessna 150          745
Cessna 172P         661
Cessna 182          611
Cessna 180          594
Piper PA-18         550
Name: count, dtype: int64


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [9]:
other_cols = ['Engine.Type', 'Weather.Condition', 'Number.of.Engines',
              'Purpose.of.flight', 'Broad.phase.of.flight']

# Inspect each column
for col in other_cols:
    print(f"--- {col} ---")
    print(data_df[col].value_counts(dropna=False), "\n")

# --- Engine.Type ---
# 'Unknown' appears as a value alongside NaN — consolidate to a single marker
data_df['Engine.Type'] = data_df['Engine.Type'].str.strip()
data_df['Engine.Type'] = data_df['Engine.Type'].replace('Unknown', np.nan)

# --- Weather.Condition ---
# 'UNK' and 'Unk' both mean unknown — standardize to 'Unknown'
data_df['Weather.Condition'] = data_df['Weather.Condition'].str.strip()
data_df['Weather.Condition'] = data_df['Weather.Condition'].replace({'UNK': 'Unknown', 'Unk': 'Unknown'})

# --- Number.of.Engines ---
# Already numeric (float64 due to NaN); no string cleaning needed
# Note: 0-engine records exist (gliders/balloons) — leave as-is, valid data
print("Number.of.Engines unique values:", sorted(data_df['Number.of.Engines'].dropna().unique()))

# --- Purpose.of.flight ---
# 'Unknown' appears as a value alongside NaN — consolidate
data_df['Purpose.of.flight'] = data_df['Purpose.of.flight'].str.strip()
data_df['Purpose.of.flight'] = data_df['Purpose.of.flight'].replace('Unknown', np.nan)

# --- Broad.phase.of.flight ---
# High missingness (31%) and 'Unknown' values — consolidate, leave NaN as-is
data_df['Broad.phase.of.flight'] = data_df['Broad.phase.of.flight'].str.strip()
data_df['Broad.phase.of.flight'] = data_df['Broad.phase.of.flight'].replace('Unknown', np.nan)

print("\nValue counts after cleaning:")
for col in other_cols:
    print(f"\n--- {col} ---")
    print(data_df[col].value_counts(dropna=False))

--- Engine.Type ---
Engine.Type
Reciprocating    54401
NaN               4292
Turbo Shaft       2896
Turbo Prop        2446
Unknown           1404
Turbo Fan         1098
Turbo Jet          311
LR                   1
UNK                  1
NONE                 1
Name: count, dtype: int64 

--- Weather.Condition ---
Weather.Condition
VMC    58541
IMC     5024
NaN     2488
UNK      614
Unk      184
Name: count, dtype: int64 

--- Number.of.Engines ---
Number.of.Engines
1.0    53984
2.0     8009
NaN     3756
0.0      725
4.0      200
3.0      176
8.0        1
Name: count, dtype: int64 

--- Purpose.of.flight ---
Purpose.of.flight
Personal                     36638
Instructional                 9278
Unknown                       4330
Aerial Application            4128
NaN                           3592
Business                      3324
Positioning                   1368
Other Work Use                1041
Aerial Observation             704
Public Aircraft                624
Ferry           

Number.of.Engines unique values: [np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(8.0)]

Value counts after cleaning:

--- Engine.Type ---
Engine.Type
Reciprocating    54401
NaN               5696
Turbo Shaft       2896
Turbo Prop        2446
Turbo Fan         1098
Turbo Jet          311
LR                   1
UNK                  1
NONE                 1
Name: count, dtype: int64

--- Weather.Condition ---
Weather.Condition
VMC        58541
IMC         5024
NaN         2488
Unknown      798
Name: count, dtype: int64

--- Number.of.Engines ---
Number.of.Engines
1.0    53984
2.0     8009
NaN     3756
0.0      725
4.0      200
3.0      176
8.0        1
Name: count, dtype: int64

--- Purpose.of.flight ---
Purpose.of.flight
Personal                     36638
Instructional                 9278
NaN                           7922
Aerial Application            4128
Business                      3324
Positioning                   1368
Other Work U

**Cleaning summary:**

| Column | Issue | Action |
|---|---|---|
| `Engine.Type` | `'Unknown'` string duplicates NaN meaning | Replaced `'Unknown'` with `NaN`; NaNs left as-is |
| `Weather.Condition` | Both `'UNK'` and `'Unk'` used for unknown | Standardized both to `'Unknown'` |
| `Number.of.Engines` | Already numeric float64; 0-engine records present | No changes — 0 engines is valid for gliders/balloons |
| `Purpose.of.flight` | `'Unknown'` string duplicates NaN meaning | Replaced `'Unknown'` with `NaN`; NaNs left as-is |
| `Broad.phase.of.flight` | 31% NaN; `'Unknown'` string also present | Replaced `'Unknown'` with `NaN`; high missingness noted but NaNs retained |

NaN values are **not imputed** in any of these columns — they may still be useful as grouping variables in EDA for the non-missing subset.

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

**Column removal decisions:**
- **Threshold: >50% missing** — columns above this level have too few valid values to be useful for grouping or modeling; imputing at this scale would introduce more noise than signal.
- **Additional drops** — columns removed regardless of missingness because they carry no analytical value for safety analysis: `Accident.Number`, `Registration.Number`, `Report.Status`, `Publication.Date`, `Airport.Code`, `Airport.Name`.

In [10]:
# Inspect % missing per column, sorted descending
missing_pct = data_df.isna().mean().sort_values(ascending=False)
print("% Missing per column:")
print((missing_pct * 100).round(1).to_string(), "\n")

# Drop columns exceeding 50% missing — too sparse to be analytically useful
threshold = 0.50
cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()
print(f"Columns above {threshold:.0%} missing threshold:")
print(cols_to_drop, "\n")

data_df = data_df.drop(columns=cols_to_drop)

# Also drop columns with no analytical value for safety analysis regardless of missingness
extra_drops = ['Accident.Number', 'Registration.Number', 'Report.Status',
               'Publication.Date', 'Airport.Code', 'Airport.Name']
# Only drop those still present (some may already be removed)
extra_drops = [c for c in extra_drops if c in data_df.columns]
data_df = data_df.drop(columns=extra_drops)

print(f"Shape after column removal: {data_df.shape}")
print("\nRemaining columns:")
print(data_df.columns.tolist())

% Missing per column:
Schedule                  87.3
Air.carrier               82.9
FAR.Description           70.3
Aircraft.Category         70.1
Longitude                 62.6
Latitude                  62.6
Airport.Code              43.3
Airport.Name              40.6
Broad.phase.of.flight     28.4
Publication.Date          17.2
Purpose.of.flight         11.9
Engine.Type                8.5
Report.Status              6.0
Number.of.Engines          5.6
Weather.Condition          3.7
Registration.Number        1.3
Injury.Rate                0.5
Injury.Severity            0.4
Country                    0.3
Location                   0.1
Total.Minor.Injuries       0.0
Total.Passengers           0.0
Is.Destroyed               0.0
Total.Uninjured            0.0
Event.Id                   0.0
Total.Serious.Injuries     0.0
Total.Fatal.Injuries       0.0
Investigation.Type         0.0
Amateur.Built              0.0
Model                      0.0
Make                       0.0
Aircraft.damage  

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [11]:
output_path = 'data/AviationData_Cleaned.csv'
data_df.to_csv(output_path, index=False)

print(f"Saved cleaned DataFrame to '{output_path}'")
print(f"Final shape: {data_df.shape}")
print(f"\nColumns saved:")
print(data_df.columns.tolist())
print(f"\nDerived columns added: 'Total.Passengers', 'Injury.Rate', 'Is.Destroyed', 'Make.Model'")

Saved cleaned DataFrame to 'data/AviationData_Cleaned.csv'
Final shape: (66851, 23)

Columns saved:
['Event.Id', 'Investigation.Type', 'Event.Date', 'Location', 'Country', 'Injury.Severity', 'Aircraft.damage', 'Make', 'Model', 'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition', 'Broad.phase.of.flight', 'Total.Passengers', 'Injury.Rate', 'Is.Destroyed', 'Make.Model']

Derived columns added: 'Total.Passengers', 'Injury.Rate', 'Is.Destroyed', 'Make.Model'
